In [1]:
import os
from os import sys
sys.path.append('./data')
import numpy as np
import matplotlib.pyplot as plt
from species import *

All variables set as e.g. krome_idx_H2
plot 'your_output' u 1:krome_idx_H2
 the offset is nkrome= 0


In [2]:
os.system('clear')
os.system('make gfortran')

0

In [3]:
def write_test(NH=1e22):
    write_test_Xray(NH=NH)
    write_test_stop(NH=NH)
    write_test0(NH=NH)

def Av(NH):
    return(str(NH*5.3e-22)+'d0')

def write_test_Xray(NH):
    with open('./test_Xray.f90', 'w') as f:
        f.write(
'program test_krome\n\n\
  use krome_main\n\
  use krome_user\n\
  use krome_user_commons\n\
  use krome_ode\n\
  use krome_getphys\n\n\
  implicit none\n\n\
  integer,parameter::nx=krome_nmols\n\
  real*8::x(nx),m(nx+4),Tgas,t,dt,spy,xH,dust2gas,x1(nx)\n\
  integer,parameter::nd=krome_ndust,imax=100\n\
  real*8::xdust(nd),adust(nd),xdusti(nd),data(imax,nd),dataT(imax)\n\
  integer::i,j,k\n\
  spy = 3600. * 24. * 365. !seconds per year\n\
  Tgas = 1d1 !gas temperature (K)\n\
  xH = 2d4 !Hydrogen density\n\n\
  !user commons for opacity and CR rate\n\
  call krome_set_user_av({}) !opacity Av (#)\n\
  call krome_set_user_crate(1.3d-17) !CR rate (1/s)\n\
  call krome_set_user_gas_dust_ratio(1d2) !gas/dust\n\
  call krome_init()\n\n\
  x(:) = 1.d-20\n\
  !initial densities (model EA2 Wakelam+Herbst 2008)\n\
  x(KROME_idx_H2)  = 0.5d0   * xH\n\
  x(KROME_idx_He)  = 9d-2   * xH\n\
  x(KROME_idx_N)   = 7.6d-5  * xH\n\
  x(KROME_idx_O)   = 2.56d-4 * xH\n\
  x(KROME_idx_Cj)  = 1.2d-4  * xH\n\
  x(KROME_idx_Sj)  = 1.5d-5  * xH\n\
  x(KROME_idx_SIj) = 1.7d-6  * xH\n\
  x(KROME_idx_Fej) = 2d-7   * xH\n\
  x(KROME_idx_Naj) = 2d-7   * xH\n\
  x(KROME_idx_Mgj) = 2.4d-6  * xH\n\
  x(KROME_idx_Clj) = 1.8d-7  * xH\n\
  x(KROME_idx_Pj)  = 1.17d-7 * xH\n\
  x(KROME_idx_Fj)  = 1.8d-8  * xH\n\n\
  !calculate elctrons (neutral cloud)\n\
  x(KROME_idx_e) = krome_get_electrons(x(:))\n\n\
  call krome(x(:),Tgas,1d8*spy)\n\
  dt = 1d0*spy !time-step (s)\n\
  t = 0 !initial time (s)\n\n\
  call krome_set_J21xray(1d0)\n\
  !output header\n\
  open(unit=77, file="./data/dis")\n\
  write(77,\'(a)\') "#time "//trim(krome_get_names_header())\n\
  x1(:)=x(:)\n\
  do\n\
    print \'(a10,E18.8,a3)\',"time:",t/spy,"yr"\n\
    call krome(x1(:),Tgas,dt) !call KROME\n\
    x1(:)=max(1d-50*xH,x1(:))\n\
    k = k + 1\n\
    t = t + dt !increase time\n\
    dt = max(dt, t/5d0) !increase time-step\n\
    write(77,\'(999E18.8)\') t/spy,x1(:)/xH\n\
    if(t>1d6*spy) exit !exit when overshoot ~1d6 years\n\
  end do\n\n\
  call krome_set_J21xray(1d0)\n\
  dt = 1d0*spy !time-step (s) renewed\n\
  t = 1d6*spy !initial time (s) renewed\n\n\
  k = 0\n\
  do\n\
    print \'(a10,E18.8,a3)\',"time:",t/spy,"yr"\n\
    call krome(x1(:),Tgas,dt) !call KROME\n\
    x1(:)=max(1d-50*xH,x1(:))\n\
    k = k + 1\n\
    t = t + dt !increase time\n\
    if (mod(k,10) == 0) call trace(nx,t/spy,x1(:))\n\
    dt = max(dt, (t-1d6*spy)/5d0) !increase time-step\n\
    write(77,\'(999E18.8)\') t/spy,x1(:)/xH\n\
    if(t>1d8*spy) exit !exit when overshoot 1d8 years\n\
  end do\n\n\
end program test_krome'.format(Av(NH)))
        
def write_test_stop(NH):
    with open('./test_stop.f90', 'w') as f:
        f.write(
'program test_krome\n\n\
  use krome_main\n\
  use krome_user\n\
  use krome_user_commons\n\
  use krome_ode\n\
  use krome_getphys\n\n\
  implicit none\n\n\
  integer,parameter::nx=krome_nmols\n\
  real*8::x(nx),m(nx+4),Tgas,t,dt,spy,xH,dust2gas,x1(nx)\n\
  integer,parameter::nd=krome_ndust,imax=100\n\
  real*8::xdust(nd),adust(nd),xdusti(nd),data(imax,nd),dataT(imax)\n\
  integer::i,j,k\n\
  spy = 3600. * 24. * 365. !seconds per year\n\
  Tgas = 1d1 !gas temperature (K)\n\
  xH = 2d4 !Hydrogen density\n\n\
  !user commons for opacity and CR rate\n\
  call krome_set_user_av({}) !opacity Av (#)\n\
  call krome_set_user_crate(1.3d-17) !CR rate (1/s)\n\
  call krome_set_user_gas_dust_ratio(1d2) !gas/dust\n\
  call krome_init()\n\n\
  x(:) = 1.d-20\n\
  !initial densities (model EA2 Wakelam+Herbst 2008)\n\
  x(KROME_idx_H2)  = 0.5d0   * xH\n\
  x(KROME_idx_He)  = 9d-2   * xH\n\
  x(KROME_idx_N)   = 7.6d-5  * xH\n\
  x(KROME_idx_O)   = 2.56d-4 * xH\n\
  x(KROME_idx_Cj)  = 1.2d-4  * xH\n\
  x(KROME_idx_Sj)  = 1.5d-5  * xH\n\
  x(KROME_idx_SIj) = 1.7d-6  * xH\n\
  x(KROME_idx_Fej) = 2d-7   * xH\n\
  x(KROME_idx_Naj) = 2d-7   * xH\n\
  x(KROME_idx_Mgj) = 2.4d-6  * xH\n\
  x(KROME_idx_Clj) = 1.8d-7  * xH\n\
  x(KROME_idx_Pj)  = 1.17d-7 * xH\n\
  x(KROME_idx_Fj)  = 1.8d-8  * xH\n\n\
  !calculate elctrons (neutral cloud)\n\
  x(KROME_idx_e) = krome_get_electrons(x(:))\n\n\
  call krome(x(:),Tgas,1d8*spy)\n\
  dt = 1d0*spy !time-step (s)\n\
  t = 0 !initial time (s)\n\n\
  call krome_set_J21xray(1d0)\n\
  !output header\n\
  open(unit=77, file="./data/dis")\n\
  write(77,\'(a)\') "#time "//trim(krome_get_names_header())\n\
  x1(:)=x(:)\n\
  do\n\
    print \'(a10,E18.8,a3)\',"time:",t/spy,"yr"\n\
    call krome(x1(:),Tgas,dt) !call KROME\n\
    x1(:)=max(1d-50*xH,x1(:))\n\
    k = k + 1\n\
    t = t + dt !increase time\n\
    dt = max(dt, t/5d0) !increase time-step\n\
    write(77,\'(999E18.8)\') t/spy,x1(:)/xH\n\
    if(t>1d6*spy) exit !exit when overshoot ~1d6 years\n\
  end do\n\n\
  call krome_set_J21xray(0d0)\n\
  dt = 1d0*spy !time-step (s) renewed\n\
  t = 1d6*spy !initial time (s) renewed\n\n\
  k = 0\n\
  do\n\
    print \'(a10,E18.8,a3)\',"time:",t/spy,"yr"\n\
    call krome(x1(:),Tgas,dt) !call KROME\n\
    x1(:)=max(1d-50*xH,x1(:))\n\
    k = k + 1\n\
    t = t + dt !increase time\n\
    if (mod(k,10) == 0) call trace(nx,t/spy,x1(:))\n\
    dt = max(dt, (t-1d6*spy)/5d0) !increase time-step\n\
    write(77,\'(999E18.8)\') t/spy,x1(:)/xH\n\
    if(t>1d8*spy) exit !exit when overshoot 1d8 years\n\
  end do\n\n\
end program test_krome'.format(Av(NH)))
        
def write_test0(NH):
    with open('./test0.f90', 'w') as f:
        f.write(
'program test_krome\n\n\
  use krome_main\n\
  use krome_user\n\
  use krome_user_commons\n\
  use krome_ode\n\
  use krome_getphys\n\n\
  implicit none\n\n\
  integer,parameter::nx=krome_nmols\n\
  real*8::x(nx),m(nx+4),Tgas,t,dt,spy,xH,dust2gas,x1(nx)\n\
  integer,parameter::nd=krome_ndust,imax=100\n\
  real*8::xdust(nd),adust(nd),xdusti(nd),data(imax,nd),dataT(imax)\n\
  integer::i,j,k\n\
  spy = 3600. * 24. * 365. !seconds per year\n\
  Tgas = 1d1 !gas temperature (K)\n\
  xH = 2d4 !Hydrogen density\n\n\
  !user commons for opacity and CR rate\n\
  call krome_set_user_av({}) !opacity Av (#)\n\
  call krome_set_user_crate(1.3d-17) !CR rate (1/s)\n\
  call krome_set_user_gas_dust_ratio(1d2) !gas/dust\n\
  call krome_init()\n\n\
  x(:) = 1.d-20\n\
  !initial densities (model EA2 Wakelam+Herbst 2008)\n\
  x(KROME_idx_H2)  = 0.5d0   * xH\n\
  x(KROME_idx_He)  = 9d-2   * xH\n\
  x(KROME_idx_N)   = 7.6d-5  * xH\n\
  x(KROME_idx_O)   = 2.56d-4 * xH\n\
  x(KROME_idx_Cj)  = 1.2d-4  * xH\n\
  x(KROME_idx_Sj)  = 1.5d-5  * xH\n\
  x(KROME_idx_SIj) = 1.7d-6  * xH\n\
  x(KROME_idx_Fej) = 2d-7   * xH\n\
  x(KROME_idx_Naj) = 2d-7   * xH\n\
  x(KROME_idx_Mgj) = 2.4d-6  * xH\n\
  x(KROME_idx_Clj) = 1.8d-7  * xH\n\
  x(KROME_idx_Pj)  = 1.17d-7 * xH\n\
  x(KROME_idx_Fj)  = 1.8d-8  * xH\n\n\
  !calculate elctrons (neutral cloud)\n\
  x(KROME_idx_e) = krome_get_electrons(x(:))\n\n\
  call krome(x(:),Tgas,1d8*spy)\n\
  dt = 1d0*spy !time-step (s)\n\
  t = 0 !initial time (s)\n\n\
  call krome_set_J21xray(0d0)\n\
  !output header\n\
  open(unit=77, file="./data/dis_inf")\n\
  write(77,\'(a)\') "#time "//trim(krome_get_names_header())\n\
  x1(:)=x(:)\n\
  do\n\
    print \'(a10,E18.8,a3)\',"time:",t/spy,"yr"\n\
    call krome(x1(:),Tgas,dt) !call KROME\n\
    x1(:)=max(1d-50*xH,x1(:))\n\
    k = k + 1\n\
    t = t + dt !increase time\n\
    dt = max(dt, t/5d0) !increase time-step\n\
    write(77,\'(999E18.8)\') t/spy,x1(:)/xH\n\
    if(t>1d6*spy) exit !exit when overshoot ~1d6 years\n\
  end do\n\n\
  call krome_set_J21xray(0d0)\n\
  dt = 1d0*spy !time-step (s) renewed\n\
  t = 1d6*spy !initial time (s) renewed\n\n\
  k = 0\n\
  do\n\
    print \'(a10,E18.8,a3)\',"time:",t/spy,"yr"\n\
    call krome(x1(:),Tgas,dt) !call KROME\n\
    x1(:)=max(1d-50*xH,x1(:))\n\
    k = k + 1\n\
    t = t + dt !increase time\n\
    if (mod(k,10) == 0) call trace(nx,t/spy,x1(:))\n\
    dt = max(dt, (t-1d6*spy)/5d0) !increase time-step\n\
    write(77,\'(999E18.8)\') t/spy,x1(:)/xH\n\
    if(t>1d8*spy) exit !exit when overshoot 1d8 years\n\
  end do\n\n\
end program test_krome'.format(Av(NH)))

In [4]:
def AGN(NH=1e22, label=0):
    with open('krome_getphys.f90', 'r') as f:
        lines = f.readlines()
        lines[3715] = 'num2col = max(ncalc,1d-40)*{}\n'.format(
            str(NH / 2e4))
    with open('krome_getphys.f90', 'w') as f:
        for l in lines:
            f.write(l)
            
    for i in range(8):
        dis = i + 1
        nH = './rate_dis/ratexH' + str(dis) + '.dat'
        nHe = './rate_dis/ratexHe' + str(dis) + '.dat'
        orderH = 'cat ' + nH + ' > ratexH.dat'
        os.system(orderH)
        orderHe = 'cat ' + nHe + ' > ratexHe.dat'
        os.system(orderHe)

        os.system('cat ./test_Xray.f90 > ./test.f90')
        os.system('make gfortran')

        os.system('./test')
        command = 'cat ./data/dis > ./data/{}dis'.format(str(label)) + str(dis)
        os.system(command)
        command = 'cat ./data/Trace > ./data/{}Trace'.format(
            str(label)) + str(dis)
        os.system(command)
        command = 'rm ./data/dis & rm ./data/Trace'
        os.system(command)
        cowsay = 'cowsay -f www I am ionized at ' + str(dis) + ' kpc'
        os.system(cowsay)

        os.system('cat ./test_stop.f90 > ./test.f90')
        os.system('make gfortran')
        os.system('./test')
        command = 'cat ./data/dis > ./data/{}dis0'.format(
            str(label)) + str(dis)
        os.system(command)
        command = 'cat ./data/Trace > ./data/{}Trace0'.format(
            str(label)) + str(dis)
        os.system(command)
        command = 'rm ./data/dis & rm ./data/Trace'
        os.system(command)
        cowsay = 'cowsay -f www I am ionized at ' + str(
            dis) + ' kpc, but have been saved at 1E6 YEARS'
        os.system(cowsay)

    command = 'rm ./data/dis ./data/Trace'.format(str(label))
    os.system(command)
    os.system('cat ./test0.f90 > ./test.f90')
    os.system('make gfortran')
    os.system('./test')
    command = 'cat ./data/dis_inf > ./data/{}dis_inf'.format(str(label))
    os.system(command)
    command = 'cat ./data/Trace > ./data/{}Trace_inf'.format(str(label))
    os.system(command)
    command = 'rm ./data/dis_inf & rm ./data/Trace'
    os.system(command)
    os.system('cowsay -f www I feel ok')

In [5]:
NHs = [1e22, np.sqrt(10)*1e22, 1e23]
for label, i in enumerate(NHs, 1):
    write_test(NH=i)
    AGN(NH=i, label=label)